In [16]:
import pandas as pd
import numpy as np

# read in static gtfs
# gtfs static file needs to be unzipped into gtfs folder

trips = pd.read_csv('./gtfs/trips.txt', low_memory=False)
routes = pd.read_csv('./gtfs/routes.txt', low_memory=False)
gtfs_stops = pd.read_csv('./gtfs/stops.txt', low_memory=False)
stop_times = pd.read_csv('./gtfs/stop_times.txt', low_memory=False, parse_dates=['arrival_time','departure_time'])
calendar = pd.read_csv('./gtfs/calendar.txt', low_memory=False)

stop_times_drop_columns = [
    'stop_headsign',
    'continuous_pickup',
    'continuous_drop_off'
]
stop_times.drop(columns=stop_times_drop_columns, inplace=True)

stop_id_lookup = gtfs_stops.loc[:,['stop_id','parent_station']]
mask = stop_id_lookup.parent_station.isna()
stop_id_lookup.loc[mask, 'parent_station'] = stop_id_lookup.loc[mask,'stop_id']

# # create gtfs static headways

trips = trips.merge(routes.loc[:,['route_id','route_type']], how='left', on=['route_id'])
stop_times = stop_times.merge(trips.loc[:,['trip_id','route_type','route_id','service_id','direction_id']], how='left', on=['trip_id'])
stop_times = stop_times.merge(stop_id_lookup.loc[:,:], how='left', on=['stop_id'])
stop_times = stop_times.merge(calendar.loc[:,['service_id','thursday']], how='left',on=['service_id'])

sub_stop_times = stop_times.loc[(stop_times.route_type < 2),:]

def time_to_seconds(time:str) -> int:
    (hour, min, sec) = time.split(":")
    return int(hour) * 3600 + int(min) * 60 + int(sec)

sub_stop_times['departure_time_sec'] = sub_stop_times.loc[:,'departure_time'].apply(time_to_seconds)
sub_stop_times = sub_stop_times.sort_values(by=['direction_id','route_id','service_id','parent_station','departure_time_sec'])
# sub_stop_times = sub_stop_times.sort_values(by=['service_id','departure_time_sec','trip_id',])

sub_stop_times['prev_departure_time_sec'] = sub_stop_times.loc[:,'departure_time_sec'].shift().where(sub_stop_times.parent_station.eq(sub_stop_times.parent_station.shift())).astype('int', errors='ignore')
sub_stop_times.dropna(axis=0, subset=['prev_departure_time_sec'], inplace=True)
sub_stop_times.loc[:,'head_way'] = sub_stop_times.loc[:,'departure_time_sec'] - sub_stop_times.loc[:,'prev_departure_time_sec']
sub_stop_times



<ipython-input-16-cc0111aab389>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_stop_times['departure_time_sec'] = sub_stop_times.loc[:,'departure_time'].apply(time_to_seconds)


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,timepoint,checkpoint_id,route_type,route_id,service_id,direction_id,parent_station,thursday,departure_time_sec,prev_departure_time_sec,head_way
984140,52451939,05:35:00,05:35:00,70047,7,0,0,0.0,aport,1,Blue,RTL322-3-Sa-01,0,place-aport,1,20100,19020.0,1080.0
984152,52451940,05:42:00,05:42:00,70047,7,0,0,0.0,aport,1,Blue,RTL322-3-Sa-01,0,place-aport,1,20520,20100.0,420.0
984164,52451941,05:50:00,05:50:00,70047,7,0,0,0.0,aport,1,Blue,RTL322-3-Sa-01,0,place-aport,1,21000,20520.0,480.0
984176,52451942,05:57:00,05:57:00,70047,7,0,0,0.0,aport,1,Blue,RTL322-3-Sa-01,0,place-aport,1,21420,21000.0,420.0
984188,52451943,06:05:00,06:05:00,70047,7,0,0,0.0,aport,1,Blue,RTL322-3-Sa-01,0,place-aport,1,21900,21420.0,480.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954958,52371133,23:25:00,23:25:00,70100,4,0,0,0.0,wlsta,1,Red,RTL322-9-Sa-01,1,place-wlsta,1,84300,83400.0,900.0
954976,52371134,23:40:00,23:40:00,70100,4,0,0,0.0,wlsta,1,Red,RTL322-9-Sa-01,1,place-wlsta,1,85200,84300.0,900.0
954994,52371135,23:55:00,23:55:00,70100,4,0,0,0.0,wlsta,1,Red,RTL322-9-Sa-01,1,place-wlsta,1,86100,85200.0,900.0
955012,52371136,24:10:00,24:10:00,70100,4,0,0,0.0,wlsta,1,Red,RTL322-9-Sa-01,1,place-wlsta,1,87000,86100.0,900.0


In [9]:
# calendar = pd.read_csv('./gtfs/calendar.txt', low_memory=False)
# calendar[(calendar.thursday == 1)]
sub_stop_times[(sub_stop_times.thursday==1)]['service_id'].unique()

array(['RTL322-3-Sa-01', 'LRV322-1-Wdy-01', 'LRV322-1-Wdy-01-CldCrlKnr',
       'LRV322-4-Wdy-01', 'RTL322-6-Sa-01', 'RTL322-6-Sa-01-OkGrvWln',
       'RTL322-9-Sa-01'], dtype=object)

In [134]:
trips

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed,route_type
0,1,BUS322-5-Wdy-02,52110769,Harvard,NaN,0,C01-1,010111,1,NaN,1-_-0,1,3
1,1,BUS322-5-Wdy-02,52110771,Harvard,NaN,0,C01-1,010111,1,NaN,1-_-0,1,3
2,1,BUS322-5-Wdy-02,52110773,Harvard,NaN,0,C01-11,010111,1,NaN,1-_-0,1,3
3,1,BUS322-5-Wdy-02,52110775,Harvard,NaN,0,C01-11,010111,1,NaN,1-_-0,1,3
4,1,BUS322-5-Wdy-02,52110777,Harvard,NaN,0,C01-11,010111,1,NaN,1-_-0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40216,Shuttle-OakGroveWellington,RTL322-6-Sa-01-OkGrvWln,52371489-OakGroveWellington1,Oak Grove (Shuttle),NaN,1,O903_-5-1-OakGroveWellington1,WellingtonToOakGrove-S,1,3.0,Shuttle-OakGroveWellington-0-1,0,3
40217,Shuttle-OakGroveWellington,RTL322-6-Sa-01-OkGrvWln,52371490-OakGroveWellington1,Oak Grove (Shuttle),NaN,1,O903_-5-1-OakGroveWellington1,WellingtonToOakGrove-S,1,3.0,Shuttle-OakGroveWellington-0-1,0,3
40218,Shuttle-OakGroveWellington,RTL322-6-Sa-01-OkGrvWln,52371491-OakGroveWellington1,Oak Grove (Shuttle),NaN,1,O903_-4-1-OakGroveWellington1,WellingtonToOakGrove-S,1,3.0,Shuttle-OakGroveWellington-0-1,0,3
40219,Shuttle-OakGroveWellington,RTL322-6-Sa-01-OkGrvWln,52371492-OakGroveWellington1,Oak Grove (Shuttle),NaN,1,O903_-7-1-OakGroveWellington1,WellingtonToOakGrove-S,1,3.0,Shuttle-OakGroveWellington-0-1,0,3


In [133]:

next_stop = sub_stop_times.sort_values(by=['trip_id','departure_time_sec'])[['direction_id','trip_id','parent_station','route_id']]
next_stop['next_trip_id'] = next_stop['trip_id'].shift(-1)
next_stop['expect_next_parent_station'] = next_stop['parent_station'].shift(-1)
# next_stop.to_csv('stop_orders_grouped.csv', index=False)
next_stop = next_stop[(~next_stop['expect_next_parent_station'].isna()) & (next_stop['route_id'] == next_stop['route_id'])]
next_stop = next_stop.groupby(by=['route_id','direction_id','parent_station'])['expect_next_parent_station'].apply(lambda x: '|'.join(x.unique())).reset_index()

next_stop

,route_id,direction_id,parent_station,expect_next_parent_station
0,Blue,0,place-aport,place-mvbcl
1,Blue,0,place-aqucl,place-state
2,Blue,0,place-bmmnl,place-sdmnl
3,Blue,0,place-bomnl,place-wondl|place-bomnl|place-orhte
4,Blue,0,place-gover,place-bomnl
...,...,...,...,...
291,Red,1,place-qnctr,place-wlsta
292,Red,1,place-shmnl,place-jfk
293,Red,1,place-smmnl,place-fldcr
294,Red,1,place-sstat,place-dwnxg


In [95]:
# gtfs_stops[(gtfs_stops.stop_id.str.contains('Forest'))]
gtfs_stops[(~gtfs_stops.platform_name.isna())&(gtfs_stops.platform_name.str.contains('Orange Line'))]
# gtfs_stops.groupby(by=['stop_name'])['stop_id'].count().reset_index()
gtfs_stops[(gtfs_stops.stop_id == 'Forest Hills-02')]


,stop_id,stop_code,stop_name,stop_desc,platform_code,platform_name,stop_lat,stop_lon,zone_id,stop_address,stop_url,level_id,location_type,parent_station,wheelchair_boarding,municipality,on_street,at_street,vehicle_type
8164,Forest Hills-02,NaN,Forest Hills,Forest Hills - Orange Line - Track 2,2,Orange Line - Track 2,42.300635,-71.114009,RapidTransit,NaN,https://www.mbta.com/stops/place-forhl,level_-1_orange_platform,0,place-forhl,1,Boston,NaN,NaN,1.0


In [18]:
import boto3
import time
import pathlib
import pickle

# pull down vehicle position file list from s3

BUCKET_NAME = 'mbta-ctd-dataplatform-dev-springboard'
FILE_LIST_FILE = pathlib.Path('gtfs_rt_vehicle_pos')
PREFIX = 'lamp/RT_VEHICLE_POSITIONS/year=2022/month=7/day='

s3 = boto3.resource('s3')

bucket = s3.Bucket(BUCKET_NAME)
client = bucket.meta.client

t0 = time.monotonic()

if FILE_LIST_FILE.is_file():
    with open(FILE_LIST_FILE, mode='rb') as f:
        obj_list = pickle.load(f)

else:
    obj_list = []
    for prefix in ('20','21','22',):
        paginator = client.get_paginator('list_objects_v2')
        pages = paginator.paginate(
            Bucket=BUCKET_NAME,
            Prefix=f'{PREFIX}{prefix}/',
        )
        for page in pages:
            for obj in page['Contents']:
                if obj['Size'] > 0:
                    obj_list.append(obj)
    with open(FILE_LIST_FILE, mode='wb') as f:
        pickle.dump(obj_list, f)

run_time = time.monotonic() - t0

print(f"{len(obj_list):,} files found in {run_time:.2f} seconds")


129 files found in 0.00 seconds


In [19]:
import io
from typing import IO

# pull down vehicle positions files from s3 and concat into dataframe

def get_zip_buffer(filename: str) -> IO[bytes]:
    """
    Get a buffer for a zip file from s3 so that it can be read by zipfile
    module. filename is assumed to be the full path to the zip file without the
    s3:// prefix. Return it along with the last modified date for this s3
    object.
    """
    # inspired by
    # https://betterprogramming.pub/unzip-and-gzip-incoming-s3-files-with-aws-lambda-f7bccf0099c9
    (bucket, file) = filename.split("/", 1)
    s3_resource = boto3.resource("s3")
    zipped_file = s3_resource.Object(bucket_name=bucket, key=file)

    return io.BytesIO(zipped_file.get()["Body"].read())


drop_columns = [
    'occupancy_percentage',
    'occupancy_status',
    'vehicle_id', # always same as 'entity_id'
    # 'start_date', # redundant with timestamp
    # 'vehicle_label',
    'vehicle_consist',
    'bearing',
    'latitude',
    'longitude',
    'trip_id', # Always None?
]
import pandas as pd

rt_df = None
for obj in obj_list:
    filename = f"{BUCKET_NAME}/{obj['Key']}"
    print(f"Downloading: {filename}")
    t_df = pd.read_parquet(get_zip_buffer(filename))
    t_df.drop(columns=drop_columns, inplace=True)
    t_df = t_df[(t_df.route_id.isin(sub_stop_times.route_id.unique()))]
    if rt_df is None:
        rt_df = t_df
    else:
        rt_df = pd.concat([rt_df,t_df])
    print(f"Processed: {filename}")

rt_df

Downloading: mbta-ctd-dataplatform-dev-springboard/lamp/RT_VEHICLE_POSITIONS/year=2022/month=7/day=20/hour=0/ade29e12f6144ebdaa27c86083a358b3-0.parquet
Processed: mbta-ctd-dataplatform-dev-springboard/lamp/RT_VEHICLE_POSITIONS/year=2022/month=7/day=20/hour=0/ade29e12f6144ebdaa27c86083a358b3-0.parquet
Downloading: mbta-ctd-dataplatform-dev-springboard/lamp/RT_VEHICLE_POSITIONS/year=2022/month=7/day=20/hour=0/eecaaf159f804e9686f4d16f816a708f-0.parquet
Processed: mbta-ctd-dataplatform-dev-springboard/lamp/RT_VEHICLE_POSITIONS/year=2022/month=7/day=20/hour=0/eecaaf159f804e9686f4d16f816a708f-0.parquet
Downloading: mbta-ctd-dataplatform-dev-springboard/lamp/RT_VEHICLE_POSITIONS/year=2022/month=7/day=20/hour=1/2e14b3724450454d8bd2578e63995dae-0.parquet
Processed: mbta-ctd-dataplatform-dev-springboard/lamp/RT_VEHICLE_POSITIONS/year=2022/month=7/day=20/hour=1/2e14b3724450454d8bd2578e63995dae-0.parquet
Downloading: mbta-ctd-dataplatform-dev-springboard/lamp/RT_VEHICLE_POSITIONS/year=2022/month=7

,feed_timestamp,entity_id,current_status,current_stop_sequence,stop_id,vehicle_timestamp,speed,direction_id,route_id,schedule_relationship,start_date,start_time,vehicle_label
0,1658275200,G-10225,IN_TRANSIT_TO,560.0,70152,1658275195,NaN,1.0,Green-D,SCHEDULED,20220719,19:34:00,3908-3915
5,1658275200,O-5471B132,IN_TRANSIT_TO,180.0,70035,1658275152,NaN,1.0,Orange,SCHEDULED,20220719,19:19:00,1313
8,1658275200,G-10282,INCOMING_AT,450.0,70187,1658275138,NaN,0.0,Green-D,SCHEDULED,20220719,19:27:00,3624-3813
32,1658275200,R-5471A548,STOPPED_AT,180.0,70070,1658275137,NaN,1.0,Red,SCHEDULED,20220719,19:17:00,1704
33,1658275200,G-10249,IN_TRANSIT_TO,450.0,70187,1658275187,NaN,0.0,Green-D,ADDED,20220719,19:39:30,3802-3620
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81997,1658483999,G-10047,INCOMING_AT,7.0,70264,1658483989,11.0,1.0,Mattapan,SCHEDULED,20220722,05:53:00,3265
82005,1658483999,G-10066,STOPPED_AT,600.0,70243,1658483932,NaN,0.0,Green-E,ADDED,20220722,05:35:04,3706-3870
82009,1658483999,R-5471C3B5,STOPPED_AT,1.0,Braintree-01,1658483142,NaN,1.0,Red,ADDED,20220722,05:45:44,1857
82027,1658483999,R-5471C3A6,STOPPED_AT,190.0,70068,1658483980,NaN,1.0,Red,ADDED,20220722,04:44:45,1711


In [37]:
# missed_stops = rt_df.sort_values(by=['entity_id','feed_timestamp'])
missed_stops = rt_df.copy()
# missed_stops.current_status != 'STOPPED_AT'
missed_stops.loc[missed_stops['current_status'] != 'STOPPED_AT','current_status'] = 'MOVING'

group_by_cols = ['entity_id','current_status','current_stop_sequence','stop_id','direction_id','route_id','schedule_relationship','start_date','start_time','vehicle_label']
aggfunc = {'vehicle_timestamp':['min','max']}
missed_stops = missed_stops.pivot_table(missed_stops, index=group_by_cols, aggfunc=aggfunc).reset_index()
missed_stops.columns = [col[0] if col[1] == '' else '_'.join(col).strip() for col in missed_stops.columns.values]

missed_stops = missed_stops.merge(stop_id_lookup, how='left', on=['stop_id'])
missed_stops['record_id'] = missed_stops['entity_id'].astype('str') + missed_stops['direction_id'].astype('str') + missed_stops['parent_station'].astype('str')

missed_stops.sort_values(by=['entity_id','vehicle_timestamp_max'])

,entity_id,current_status,current_stop_sequence,stop_id,direction_id,route_id,schedule_relationship,start_date,start_time,vehicle_label,vehicle_timestamp_max,vehicle_timestamp_min,parent_station,record_id
6,B-5471B5C1,MOVING,1.0,70059,0.0,Blue,SCHEDULED,20220719,20:04:00,0743,1658275288,1658275252,place-wondl,B-5471B5C10.0place-wondl
155,B-5471B5C1,STOPPED_AT,1.0,70059,0.0,Blue,SCHEDULED,20220719,20:04:00,0743,1658275303,1658275303,place-wondl,B-5471B5C10.0place-wondl
23,B-5471B5C1,MOVING,10.0,70057,0.0,Blue,SCHEDULED,20220719,20:04:00,0743,1658275475,1658275456,place-rbmnl,B-5471B5C10.0place-rbmnl
167,B-5471B5C1,STOPPED_AT,10.0,70057,0.0,Blue,SCHEDULED,20220719,20:04:00,0743,1658275492,1658275492,place-rbmnl,B-5471B5C10.0place-rbmnl
35,B-5471B5C1,MOVING,20.0,70055,0.0,Blue,SCHEDULED,20220719,20:04:00,0743,1658275574,1658275533,place-bmmnl,B-5471B5C10.0place-bmmnl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200972,R-5471C5D8,STOPPED_AT,150.0,70089,0.0,Red,ADDED,20220722,15:55:00,1642,1658523731,1658523731,place-fldcr,R-5471C5D80.0place-fldcr
200938,R-5471C5D8,MOVING,160.0,70091,0.0,Red,ADDED,20220722,15:55:00,1642,1658523888,1658523819,place-smmnl,R-5471C5D80.0place-smmnl
200974,R-5471C5D8,STOPPED_AT,160.0,70091,0.0,Red,ADDED,20220722,15:55:00,1642,1658523914,1658523914,place-smmnl,R-5471C5D80.0place-smmnl
200940,R-5471C5D8,MOVING,170.0,70093,0.0,Red,ADDED,20220722,15:55:00,1642,1658524007,1658523968,place-asmnl,R-5471C5D80.0place-asmnl


In [20]:
group_by_cols = ['entity_id','current_status','current_stop_sequence','stop_id','direction_id','route_id','schedule_relationship','start_date','start_time','vehicle_label']


aggfunc = {'vehicle_timestamp':['min','max']}
rt_head = rt_df.pivot_table(rt_df, index=group_by_cols, aggfunc=aggfunc).reset_index()
rt_head.columns = [col[0] if col[1] == '' else '_'.join(col).strip() for col in rt_head.columns.values]

rt_head = rt_head[(rt_head.current_status == 'STOPPED_AT')]
rt_head = rt_head.merge(stop_id_lookup, how='left', on=['stop_id']).sort_values(by=['direction_id','start_date','route_id','parent_station','vehicle_timestamp_max'])

rt_head = rt_head.drop_duplicates(subset=['entity_id','vehicle_label','parent_station','vehicle_timestamp_min'])

rt_head['prev_departure_time_sec'] = rt_head['vehicle_timestamp_max'].shift().where(rt_head.parent_station.eq(rt_head.parent_station.shift()))


# rt_head.prev_departure_time_sec.fillna(0, inplace=True)
# rt_head['prev_departure_time_sec'] = rt_head['prev_departure_time_sec'].astype('int', errors='ignore')
# rt_head['head_way'] = rt_head['vehicle_timestamp_max'] - rt_head['prev_departure_time_sec']
# rt_head[(rt_head.route_id == 'Orange')].to_csv('rt_headways_oragne.csv', index=False)


rt_head.dropna(axis=0, subset=['prev_departure_time_sec'], inplace=True)
rt_head['prev_departure_time_sec'] = rt_head['prev_departure_time_sec'].astype('int', errors='ignore')
rt_head['head_way'] = rt_head['vehicle_timestamp_max'] - rt_head['prev_departure_time_sec']
rt_head['dt'] = pd.to_datetime(rt_head['vehicle_timestamp_min'], unit='s')

rt_head

,entity_id,current_status,current_stop_sequence,stop_id,direction_id,route_id,schedule_relationship,start_date,start_time,vehicle_label,vehicle_timestamp_max,vehicle_timestamp_min,parent_station,prev_departure_time_sec,head_way,dt
72,B-5471B5C1,STOPPED_AT,60.0,70047,0.0,Blue,SCHEDULED,20220719,20:04:00,0743,1658276003,1658276003,place-aport,1658275521,482,2022-07-20 00:13:23
483,B-5471B5E7,STOPPED_AT,60.0,70047,0.0,Blue,SCHEDULED,20220719,20:13:00,0770,1658276561,1658276561,place-aport,1658276003,558,2022-07-20 00:22:41
620,B-5471B5E8,STOPPED_AT,60.0,70047,0.0,Blue,SCHEDULED,20220719,20:22:00,0711,1658277110,1658277110,place-aport,1658276561,549,2022-07-20 00:31:50
186,B-5471B5C2,STOPPED_AT,60.0,70047,0.0,Blue,SCHEDULED,20220719,20:30:00,0783,1658277816,1658277816,place-aport,1658277110,706,2022-07-20 00:43:36
295,B-5471B5CD,STOPPED_AT,60.0,70047,0.0,Blue,SCHEDULED,20220719,20:39:00,0791,1658278301,1658278301,place-aport,1658277816,485,2022-07-20 00:51:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94319,R-5471BE8A,STOPPED_AT,30.0,70100,1.0,Red,ADDED,20220722,18:24:17,1607,1658531349,1658531349,place-wlsta,1658529979,1370,2022-07-22 23:09:09
90314,R-5471BAC4,STOPPED_AT,30.0,70100,1.0,Red,ADDED,20220722,19:04:28,1500,1658531825,1658531825,place-wlsta,1658531349,476,2022-07-22 23:17:05
99442,R-5471C3B5,STOPPED_AT,30.0,70100,1.0,Red,ADDED,20220722,19:18:51,1880,1658532527,1658532527,place-wlsta,1658531825,702,2022-07-22 23:28:47
96590,R-5471BEEB,STOPPED_AT,30.0,70100,1.0,Red,SCHEDULED,20220722,19:31:00,1848,1658533290,1658533290,place-wlsta,1658532527,763,2022-07-22 23:41:30


In [13]:
import matplotlib.pyplot as plt

start_time = 1658395804

for route_id in sorted(rt_head.route_id.unique()):
    route_df = rt_head[
            (rt_head.route_id == route_id)
            & (rt_head.vehicle_timestamp_min > start_time)
            & (rt_head.vehicle_timestamp_min < start_time + 70000)
            # & (rt_head.head_way < 2000)
        ]
    sch_df = sub_stop_times[
        (sub_stop_times.route_id == route_id)
        & (sub_stop_times.thursday == 1)
    ]
    for direction_id in (0,1):
        t_df = route_df[
            (route_df.direction_id == direction_id)
        ]
        t2_df = sch_df[
            (sch_df.direction_id == direction_id)
        ]
        fig, ax = plt.subplots(figsize=(20,8))
        for checkpoint_id in sorted(t_df.parent_station.unique()):
            line_df = t_df[(t_df.parent_station == checkpoint_id)]
            ax.scatter(line_df['dt'], line_df['head_way'], label=checkpoint_id )
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        ax.set_xlabel('departure_time (seconds)')
        ax.set_ylabel('headway (seconds)')
        ax.set_title(f'Actual {route_id} Line - headways - direction {direction_id}')

        fig, ax = plt.subplots(figsize=(20,8))
        for checkpoint_id in sorted(t2_df.parent_station.unique()):
            line_df = t2_df[(t2_df.parent_station == checkpoint_id)]
            ax.scatter(line_df['departure_time_sec'], line_df['head_way'], label=checkpoint_id )
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        ax.set_xlabel('departure_time (seconds)')
        ax.set_ylabel('headway (seconds)')
        ax.set_title(f'Expected {route_id} Line - headways - direction {direction_id}')

NameError: name 'rt_head' is not defined

In [121]:





# stops['next_stop_id_grouped'] = stops['stop_id_grouped'].shift(-1)
# stops
# stops = stops.merge(next_stop, how='left', on=['route_id','stop_id_grouped','direction_id'])

# stops['next_stop_ok'] = np.where((stops['next_stop_id_grouped'] == stops['expect_next_stop_id']), True,False)
# stops[(stops.expect_next_stop_id.isna())].groupby(by=['route_id','stop_id_grouped']).size()
# stops.groupby(by=['next_stop_ok']).size()
# stops[(stops.next_stop_ok == False)]
# # stops

(99835, 12)


In [ ]:
for each in stops[(stops.next_stop_id_grouped.isna())].stop_id.unique():
    print(each)